Chen, Joy | May 2019 | Stat 418

# International Airfare Pricing and FX Arbitrage

### Description
An analysis of international airfare prices from the U.S. to various destinations Europe. Data is collected via webscraping using BeautifulSoup. Data and analysis will be used to construct predictive logistic model that will assist with airfare purchasing decisions.

### Acknowledgements
Courtesy of Norwegian Airlines

***
### Setup: Load Packages

In [3]:
import requests
from bs4 import BeautifulSoup

import urllib.parse
from urllib.parse import urlparse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import OrderedDict

import time
import datetime
import pytz
import re

pd.options.display.max_rows = 50

***

### Setting Airports, Currency and Date Options
Prior to extracting data, let's define our date range for flights as well as airports and currencies we are interested in  

##### AIRPORTS
**US**: Los Angeles (LAX) | Oakland (OAK) | New York-JFK (JFK)  
**EU**: Copenhagen (CPH) | Stockholm, Sweden (ARN) | Paris (CDG) | London-Gatwick (LGW) | Barcelona (BCN)

In [4]:
# Create lists of origin and destination airports
orig_port = ['LAX','OAK','JFK']
dest_port = ['CPH','ARN','CDG','LGW','BCN']

# Create lists of all possible flight itinerary combinations
orig_port_seq = list(np.repeat(orig_port,len(dest_port)))
dest_port_seq = dest_port * len(orig_port)

##### CURRENCIES

US Dollar (USD), Euro (EUR), Great Britian Pound (GBP), Swedish Krona (SEK)

In order to get value of currency, we must access website on different domains  
i.e. USD = US website, EUR = French website, GBP = UK website, SEK = Swedish website

In [5]:
# Define currencies
curr = {'USD': 'us', 'GBP': 'uk', 'EUR': 'fr', 'SEK': 'se'}

# NOTE: We will use the US website as our base to extract flight info
usd_index = list(curr).index('USD')

##### DATES & TIMES

In [6]:
# Specify time right now
today_date = datetime.datetime.now(pytz.timezone('US/Pacific'))

***

### Extract Data: Run Webscraper

In [7]:
def get_tix(num_days):

    # SET TIMES
    # Create list of dates beginning from now
    dates = pd.date_range(today_date.strftime('%Y-%m-%d'), periods = num_days).tolist()



    # CREATE MASTER DATAFRAME OBJECT
    tix_data = pd.DataFrame()        


    #-------------------------------

    # RUN SCRAPER

    for n in range(0, len(orig_port_seq),1):   # to run scraper through all airport combinations
        print(orig_port_seq[n], 'to', dest_port_seq[n])

        for x in range(0,len(dates),1):        # ...to run each airport pair through all dates

            print(dates[x].strftime('%Y-%m-%d'))
            print('Retreiving...')

            #--------------------------
            # [1] CREATE URLS, REQUEST HTMLS, CREATE SOUP

            url = []
            tix_raw = []
            tix_soup = []

            for i in range(0, len(curr), 1):

                # 1.1 Create multiple urls corresponding to different currencies desired
                url.append(
                    str('https://www.norwegian.com/%s/ipc/availability/avaday?A_City=%s&TripType=1&D_City=%s&D_Month=%s%s&D_Day=%s'
                        % (list(curr.values())[i],
                           dest_port_seq[n], orig_port_seq[n], 
                           dates[x].strftime('%Y'),dates[x].strftime("%m"),dates[x].strftime("%d"))
                       )
                )

                # SLEEP REQUESTS
                print('Sleeping...')
                time.sleep(4)

                # 1.2 Request pages
                print('Requesting: ', list(curr.keys())[i], dates[x].strftime("%Y-%m-%d"))
                tix_raw.append(requests.get(url[i]).text)

                # 1.3 Turn into soup
                tix_soup.append(BeautifulSoup(tix_raw[i], 'html.parser'))
                print('Parsed to soup!')

            #--------------------------
            # [2] EXTRACT RELEVANT FIELDS
            # NOTE: Norwegian has two different site structures, any code with tag <ADDRESSING STRUC CHANGE> refers to this

            #-----

            # 2.1 Total Number of Daily Flights
            # This is calculated by how many flight duration values are present (1:1 mapping)
            duration_all = [i.text for i in tix_soup[usd_index].find_all('td', class_="duration")]

            remove = ['Direct','stop']
            duration_filtered = [i for i in duration_all if not any(word in i.split(' ') for word in remove)]
            duration = [i.split(': ', 1)[1] for i in duration_filtered]

            total_num_flights = len(duration)

            #-----

            # <ADDRESSING STRUC CHANGE> 
            # If site structure is different, alternative stop info is extracted using this 
            stops_alt = [i for i in duration_all if any(word in i.split(' ') for word in remove)]

            #-----

            # 2.2 Other Fields Based on Flight Availability
            # Addresses situation where there are no flights
            if total_num_flights == 0:
                print('No flights on', dates[x].strftime('%Y-%m-%d'))
                continue

            else:

                # 2.2.1 Dates / Times
                now = datetime.datetime.now(pytz.timezone('US/Pacific'))
                data_extract_time = [now.strftime('%y-%m-%d %H:%M:%S')] * total_num_flights
                depart_date_comb = [dates[x].strftime('%Y-%m-%d')] * total_num_flights


                # 2.2.2 Airport Orig / Dest
                orig_port_code = [orig_port_seq[n]] * total_num_flights
                dest_port_code = [dest_port_seq[n]] * total_num_flights


                # <ATTACH VARIABLES TO DATA FRAME>
                tix_tempdata = pd.concat([pd.Series(data_extract_time, name = "data_extract_time"), 
                                          pd.Series(orig_port_code, name = 'orig_port_code'),
                                          pd.Series(dest_port_code, name = 'dest_port_code'),
                                          pd.Series(depart_date_comb, name = 'depart_date')
                                         ],
                                         axis = 1)

                #-----

                # 2.2.3 Departure Info
                depart_info = [i.text for i in tix_soup[usd_index].find_all('td', class_="depdest")]

                if len(stops_alt) > 0:
                    depart_time = depart_info[0::2]
                else:
                    depart_time = depart_info[0::3]


                # 2.2.3 Arrival Info
                arrive_info = [i.text for i in tix_soup[usd_index].find_all('td', class_="arrdest")]
                arrive_time = arrive_info[0::2]


                # <ATTACH VARIABLES TO DATA FRAME>
                tix_tempdata = pd.concat([tix_tempdata, 
                                          pd.Series(depart_time, name = 'depart_time'), 
                                          pd.Series(arrive_time, name = 'arrive_time')
                                         ], 
                                         axis = 1)

                #-----

                # 2.2.4 Flight Durations
                # NOTE: duration was extracted earlier
                duration_hrs = [i.split(' ', 1)[0] for i in duration]
                duration_hrs = [float(i.split('h', 1)[0]) for i in duration_hrs]

                duration_min = [i.split(' ', 1)[1] for i in duration]
                duration_min = [float(i.split('m', 1)[0]) for i in duration_min]

                duration_total_min = np.add([x*60 for x in duration_hrs], duration_min)


                # <ATTACH VARIABLES TO DATA FRAME>
                tix_tempdata = pd.concat([tix_tempdata, 
                                          pd.Series(duration_hrs, name = 'duration_hrs'), 
                                          pd.Series(duration_min, name = 'duration_min'), 
                                          pd.Series(duration_total_min, name = 'duration_total_min')
                                         ], 
                                         axis = 1)

                #-----

                # 2.2.3 Stops / Layovers
                if len(stops_alt) > 0:
                    # If site structure changed use alt
                    stops = stops_alt[:]
                else:    
                    # Otherwise, use the usual   
                    stops = depart_info[2::3]

                # Number of stops
                stops = [i.split(' ', 1)[0] for i in stops]
                stops = [0 if i=='Direct' else i for i in stops]
                stops = [int(i) for i in stops]

                # Stop details
                stops_info = [i.text for i in tix_soup[usd_index].find_all('li', class_="tooltipclick TooltipBoxTransit")] 

                stops_time_temp = [i.split(')', 1)[0] for i in stops_info]
                stops_time_temp = [i.split('(', 1)[1] for i in stops_time_temp]
                stops_loc_temp = [i.split('in ', 1)[1] for i in stops_info]

                # Clean up to account for flights without stops
                stops_time = []
                stops_loc = []

                for i in range(0, len(stops), 1):
                    if stops[i] == 0:
                        stops_time.append(np.nan)
                        stops_loc.append(np.nan)
                    else:
                        stops_time.append(stops_time_temp[0])
                        stops_time_temp.pop(0)

                        stops_loc.append(stops_loc_temp[0])
                        stops_loc_temp.pop(0)

                # <ATTACH VARIABLES TO DATAFRAME>
                tix_tempdata = pd.concat([tix_tempdata,
                                          pd.Series(stops, name = 'stops'), 
                                          pd.Series(stops_loc, name = 'stops_loc'), 
                                          pd.Series(stops_time, name = 'stops_time')
                                         ], 
                                         axis = 1)

                #-----

                # 2.2.4 Prices

                prices_all = []
                prem_avail = []
                prices_all_filtered = []


                for i in range(0, len(curr), 1):      
                    # Extract soup for each currency
                    prices_all.append([u.text for u in tix_soup[i].find_all('td', class_ = re.compile('.*fare.*'))])

                    # Strip out any trailing or leading spaces
                    prices_all[i] =[u.strip(' ') for u in prices_all[i]]

                    # Determine if premium seats are available
                    prem_avail.append(int('Premium' in prices_all[i]))

                    # Strip out elements with words
                    remove = ['','\xa0','LowFare','LowFare+','Premium','Only','Bara','Seulement']
                    prices_all_filtered.append([u for u in prices_all[i] if not any(word in u.split(' ') for word in remove)])
                    prices_all_filtered[i] = pd.Series(
                        [u.replace('\xa0','') for u in prices_all_filtered[i]]).replace('-', np.nan).replace('[A-Za-z]', 0, regex=True)


                # Need to do some number format editing for EUR and SEK
                prices_all_filtered[list(curr).index('EUR')] = prices_all_filtered[list(curr).index('EUR')].replace(',', '.', regex=True)
                prices_all_filtered[list(curr).index('SEK')] = prices_all_filtered[list(curr).index('SEK')].replace('[.]', '', regex=True)

                # Convert values to float
                prices_all_clean = [list(u.replace(',', '',regex=True).astype(float)) for u in prices_all_filtered]

                # Determine how many columns
                ncol = [max(prem_avail[i]*5,3) for i in prem_avail]

                # Create column names
                price_types = ['prices_lowfare_','prices_lowfareplus_',
                   'prices_flex_','prices_prem_','prices_premflex_'] * len(curr.keys())

                price_curr = list(np.repeat(list(curr.keys()),len(set(price_types))))

                # Create labels for columns
                price_labels = [u+str(w) for u,w in zip(price_types, price_curr)]
                price_labels = [price_labels[i:i+5] for i in range(0, len(price_labels), 5)]  


                # Put prices into dataframe
                price_data = pd.DataFrame()

                for i in range(0, len(prices_all_clean), 1):

                    price_data = pd.concat([price_data,
                                            pd.DataFrame(np.array(prices_all_clean[i]
                                                             ).reshape(int(len(prices_all_clean[i])/ncol[i]), ncol[i]), 
                                                         columns = price_labels[i][0:ncol[i]]
                                                        )
                                           ],
                                           axis = 1)

                    # If the webpage structure excludes premium, we need to fill in those columns with NaN
                    if prem_avail[i] == 0:
                        price_data = pd.concat([price_data,
                                                pd.Series([np.nan] * total_num_flights, name = price_labels[i][3]), 
                                                pd.Series([np.nan] * total_num_flights, name = price_labels[i][4])
                                               ],
                                               axis = 1)


                # <ATTACH VARIABLES TO DATAFRAME>    
                tix_tempdata2 = pd.concat([tix_tempdata, price_data], axis=1)

                #-----

                # 2.2.5 Flight ids   
                # Find all available flight numbers and clean up
                id_all = list(OrderedDict.fromkeys(
                    [i for i in str(tix_soup[usd_index].find_all('input', class_="radio-ajax")).split("|") if i.startswith('D')])
                             )

                # Separate first and second leg ids (if applicable)
                id_leg1_temp = [value[:6].upper() for value in id_all]
                id_leg1_temp = list(pd.Series(id_leg1_temp).replace('', np.nan))

                id_leg2_temp = [value[12:len(value)-6].upper() for value in id_all]
                id_leg2_temp = list(pd.Series(id_leg2_temp).replace('', np.nan))

                id_leg1 = []
                id_leg2 = []

                # Create final leg id vectors taking into account any sold-out flights (no flight ids available)

                for i in range(0,total_num_flights,1):
                    if sum(tix_tempdata2.iloc[i,28:32].dropna()) == 0:
                        id_leg1.append(np.nan)
                        id_leg2.append(np.nan)
                    else:
                        id_leg1.append(id_leg1_temp[0])
                        id_leg1_temp.pop(0)

                        id_leg2.append(id_leg2_temp[0])
                        id_leg2_temp.pop(0)


                # Add flight ids to tempdata
                tix_tempdata2.insert(1, 'id_leg1', pd.Series(id_leg1))
                tix_tempdata2.insert(11, 'id_leg2', pd.Series(id_leg2))

            #--------------------------
            # [3] CONCATENATE TO MASTER

                tix_data = pd.concat([tix_data, tix_tempdata2])
                print('Data saved!')
    return tix_data

In [8]:
data = get_tix(1)

LAX to CPH
2019-05-29
Retreiving...
Sleeping...
Requesting:  USD 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  GBP 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  EUR 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  SEK 2019-05-29
Parsed to soup!
Data saved!
LAX to ARN
2019-05-29
Retreiving...
Sleeping...
Requesting:  USD 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  GBP 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  EUR 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  SEK 2019-05-29
Parsed to soup!
Data saved!
LAX to CDG
2019-05-29
Retreiving...
Sleeping...
Requesting:  USD 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  GBP 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  EUR 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  SEK 2019-05-29
Parsed to soup!
Data saved!
LAX to LGW
2019-05-29
Retreiving...
Sleeping...
Requesting:  USD 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  GBP 2019-05-29
Parsed to soup!
Sleeping...
Requesting:  EUR 2019-05

,data_extract_time,id_leg1,orig_port_code,dest_port_code,depart_date,depart_time,arrive_time,duration_hrs,duration_min,duration_total_min,...,prices_lowfare_EUR,prices_lowfareplus_EUR,prices_flex_EUR,prices_prem_EUR,prices_premflex_EUR,prices_lowfare_SEK,prices_lowfareplus_SEK,prices_flex_SEK,prices_prem_SEK,prices_premflex_SEK
0,19-05-29 16:13:48,DY7084,LAX,CPH,2019-05-29,19:40,17:05 +1,12.0,25.0,745.0,...,925.41,995.41,984.13,1256.37,1315.09,9936.0,10486.0,10568.0,13498.0,14130.0
1,19-05-29 16:13:48,DI7096,LAX,CPH,2019-05-29,19:30,18:15 +1,13.0,45.0,825.0,...,1116.50,1186.50,1252.61,1339.74,1475.85,12001.0,12551.0,13466.0,14404.0,15869.0
2,19-05-29 16:13:48,DY7084,LAX,CPH,2019-05-29,19:40,18:35 +1,13.0,55.0,835.0,...,925.41,995.41,984.13,1256.37,1315.09,9936.0,10486.0,10568.0,13498.0,14130.0
3,19-05-29 16:13:48,DY7116,LAX,CPH,2019-05-29,22:00,00:40 +2,17.0,40.0,1060.0,...,1023.95,1093.95,1324.48,1473.95,1774.48,11555.0,12105.0,14789.0,17732.0,20966.0
4,19-05-29 16:13:48,DI7096,LAX,CPH,2019-05-29,19:30,23:35 +1,19.0,5.0,1145.0,...,1116.50,1186.50,1252.61,1339.74,1475.85,12001.0,12551.0,13466.0,14404.0,15869.0
0,19-05-29 16:14:32,DY7084,LAX,ARN,2019-05-29,19:40,17:30 +1,12.0,50.0,770.0,...,904.06,974.06,941.43,1235.02,1272.39,9531.0,10081.0,9881.0,13093.0,13443.0
1,19-05-29 16:14:32,DY7084,LAX,ARN,2019-05-29,19:40,19:00 +1,14.0,20.0,860.0,...,872.03,942.03,941.43,1202.99,1272.39,9231.0,9781.0,9881.0,12793.0,13443.0
2,19-05-29 16:14:32,DI7096,LAX,ARN,2019-05-29,19:30,20:20 +1,15.0,50.0,950.0,...,1170.80,1240.80,1181.26,1394.04,1404.50,12389.0,12939.0,12489.0,14792.0,14892.0
3,19-05-29 16:14:32,DY7084,LAX,ARN,2019-05-29,19:40,20:30 +1,15.0,50.0,950.0,...,872.03,942.03,941.43,1202.99,1272.39,9231.0,9781.0,9881.0,12793.0,13443.0
4,19-05-29 16:14:32,DY7098,LAX,ARN,2019-05-29,22:15,00:40 +2,17.0,25.0,1045.0,...,1023.09,1093.09,1085.23,NaN,NaN,12369.0,12919.0,12969.0,NaN,NaN


In [116]:
data.to_csv('tix_data20190529.csv')